In [5]:
import tensorflow as tf
import os # for doing directory operations 
import pandas as pd # for some simple data analysis (right now, just to load in the labels data and quickly reference it)
import cv2
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
import pydicom
from tensorflow.keras.models import load_model


In [6]:
#returns the hounse fied unit of slices 
def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)


#normalize data withiin range -200 500
MIN_BOUND = -200.0
MAX_BOUND = 500.0    
def normalize(image):
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image>1] = 1.
    image[image<0] = 0.
    return image


#resample the image to 1mm in all diminsions 
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = np.array([scan[0].SliceThickness, scan[0].PixelSpacing[0], scan[0].PixelSpacing[1]], dtype=np.float32)

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
   
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')
    image = image.atype(np.float32)image = image.astype(np.float32)
    
    
    return image



<class 'numpy.ndarray'>


In [ ]:
def tumor_segm(tumur_count = [7,1,1,1,1,3,1,1,1,1,1,1,2,1,1]):
    tumor_segm = []
    pathes = []
    for i in range(15):
        s1 = "3Dircadb1/3Dircadb1."
        s2 = "/MASKS_DICOM/livertumor0"
        if tumur_count[i] == 1:
            pathes.append([s1+str(i+1)+s2+'1'])
        else:
            l = []
            for k in range(tumur_count[i]):
                l.append(s1+str(i+1)+s2+str(k+1))
            pathes.append(l)
    #print(pathes)
    
    
    tumor_slices = []
    tumor_norm = []
    for path in pathes:
    #     labels_df.get_value(patient, 'cancer')
        if len(path) == 1:
            # a couple great 1-liners from: https://www.kaggle.com/gzuidhof/data-science-bowl-2017/full-preprocessing-tutorial
            slices = [pydicom.read_file(path[0]+"/" + s) for s in os.listdir(path[0])]
            slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
            tumor_slices.append(slices)
            hf_3d = get_pixels_hu(slices)
            norm = normalize2(hf_3d)
            tumor_norm.append(norm)
            
        else:
            temp = []
            slices1 = [pydicom.read_file(path[0]+"/" + s) for s in os.listdir(path[0])]
            slices1.sort(key = lambda x: int(x.ImagePositionPatient[2]))
            dumy = get_pixels_hu(slices1)
            add = np.zeros(shape = dumy.shape)
            for k in range(len(path)):
                slices = [pydicom.read_file(path[k]+"/" + s) for s in os.listdir(path[k])]
                slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
                temp.append(slices)
                hf_3d = get_pixels_hu(slices)
                norm = normalize2(hf_3d)
                add = add + norm
            tumor_slices.append(temp)
            tumor_norm.append(add)
    
    
    return tumor_norm,tumor_slices

normalized_tumor,tumor_slices = tumor_segm()


tumors_resampled = []
for i in range(1):
    pix_resampled, spacing = resample( normalized_tumor[i], tumor_slices, [1,1,1])
    tumors_resampled.append(pix_resampled)


In [ ]:
np.save('offline-pattients.npy',tumors_resampled)